# Wprowadzenie do uczenia maszynowego w języku Julia

## Dlaczego Julia

W trakcie kursu pracować będziemy w języku [Julia](https://julialang.org/). Dlaczego?

- Język skryptowy (jak Python  czy R)
- Szybkość  (jak C)
- Silny  system typów 
- Wbudowane  zrównoleglanie  obliczeń
- Łatwość  integracji  integracji (Python, R, C, …) 


Zacznijmy od omówienia prostego przykładu. Interesuje nas oszacowanie prostego modelu regresji liniowej na [danych dotyczących cen domów w Bostonie](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html). Zacznijmy od wczytania danych:

In [1]:
using DelimitedFiles

In [2]:
isfile("housing.data") ||
  download("https://raw.githubusercontent.com/MikeInnes/notebooks/master/housing.data",
           "housing.data")
rawdata = readdlm("housing.data")'

14×505 LinearAlgebra.Adjoint{Float64,Array{Float64,2}}:
   0.02731    0.02729    0.03237  …    0.06076    0.10959    0.04741
   0.0        0.0        0.0           0.0        0.0        0.0    
   7.07       7.07       2.18         11.93      11.93      11.93   
   0.0        0.0        0.0           0.0        0.0        0.0    
   0.469      0.469      0.458         0.573      0.573      0.573  
   6.421      7.185      6.998    …    6.976      6.794      6.03   
  78.9       61.1       45.8          91.0       89.3       80.8    
   4.9671     4.9671     6.0622        2.1675     2.3889     2.505  
   2.0        2.0        3.0           1.0        1.0        1.0    
 242.0      242.0      222.0         273.0      273.0      273.0    
  17.8       17.8       18.7      …   21.0       21.0       21.0    
 396.9      392.83     394.63        396.9      393.45     396.9    
   9.14       4.03       2.94          5.64       6.48       7.88   
  21.6       34.7       33.4          23.9     

In [3]:
X = rawdata[1:13,:];
y = rawdata[14:14,:];

Obejrzyjmy wczytane dane, wykorzystamy do tego bibliotekę [<tt>DataFrames</tt>](https://juliadata.github.io/DataFrames.jl/stable/man/getting_started.html):

In [4]:
using DataFrames

In [5]:
df = DataFrames.DataFrame(vcat(X,y)')
names!(df,
    [:CRIM #- per capita crime rate by town
    :ZN #- proportion of residential land zoned for lots over 25,000 sq.ft.
    :INDUS #- proportion of non-retail business acres per town.
    :CHAS #- Charles River dummy variable (1 if tract bounds river; 0 otherwise)
    :NOX #- nitric oxides concentration (parts per 10 million)
    :RM #- average number of rooms per dwelling
    :AGE #- proportion of owner-occupied units built prior to 1940
    :DIS #- weighted distances to five Boston employment centres
    :RAD #- index of accessibility to radial highways
    :TAX #- full-value property-tax rate per $10,000
    :PTRATIO #- pupil-teacher ratio by town
    :B #- 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
    :LSTAT #- % lower status of the population
    :MEDV]) #- Median value of owner-occupied homes in $1000's)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.9,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.9,5.33,36.2
5,0.02985,0.0,2.18,0.0,0.458,6.43,58.7,6.0622,3.0,222.0,18.7,394.12,5.21,28.7
6,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.6,12.43,22.9
7,0.14455,12.5,7.87,0.0,0.524,6.172,96.1,5.9505,5.0,311.0,15.2,396.9,19.15,27.1
8,0.21124,12.5,7.87,0.0,0.524,5.631,100.0,6.0821,5.0,311.0,15.2,386.63,29.93,16.5
9,0.17004,12.5,7.87,0.0,0.524,6.004,85.9,6.5921,5.0,311.0,15.2,386.71,17.1,18.9


In [6]:
describe(df)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Float64,Float64,Float64,Float64,Nothing,Nothing,DataType
1,CRIM,3.62067,0.00906,0.25915,88.9762,,,Float64
2,ZN,11.3505,0.0,0.0,100.0,,,Float64
3,INDUS,11.1543,0.46,9.69,27.74,,,Float64
4,CHAS,0.0693069,0.0,0.0,1.0,,,Float64
5,NOX,0.554728,0.385,0.538,0.871,,,Float64
6,RM,6.28406,3.561,6.208,8.78,,,Float64
7,AGE,68.5816,2.9,77.7,100.0,,,Float64
8,DIS,3.79446,1.1296,3.1992,12.1265,,,Float64
9,RAD,9.56634,1.0,5.0,24.0,,,Float64


In [7]:
first(df, 5)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.9,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.9,5.33,36.2
5,0.02985,0.0,2.18,0.0,0.458,6.43,58.7,6.0622,3.0,222.0,18.7,394.12,5.21,28.7


Zdefiniujmy funkcję estymującą parametry regresji jako iloczyn macierzy:
$\hat{\beta} = (X^TX)^{-1}X^Ty $

In [8]:
function least_squares(X::Array{Float64},
    y::Array{Float64} , intercept::Bool = true)
    intercept ? (X = hcat(X, ones(size(X)[1]))) : X = X
    return (X'*X)^-1*X'*y
end

least_squares (generic function with 2 methods)

In [9]:
@time θ = least_squares(Matrix(X'),Array(y'))

  0.756902 seconds (2.06 M allocations: 98.678 MiB, 5.22% gc time)


14×1 Array{Float64,2}:
  -0.1074169930843746   
   0.04612073497018482  
   0.01426897099166205  
   2.6711078224329903   
 -17.633641354372656    
   3.794306798163098    
   0.0010761613071691364
  -1.4791794270368688   
   0.30153412489426484  
  -0.01205343421855803  
  -0.9588735921957785   
   0.009305401505805454 
  -0.5275996173444484   
  36.63494146667753     

In [10]:
using BenchmarkTools

In [11]:
@benchmark θ = least_squares(Matrix(X'),Matrix(y'))

BenchmarkTools.Trial: 
  memory estimate:  181.45 KiB
  allocs estimate:  18
  --------------
  minimum time:     62.060 μs (0.00% GC)
  median time:      69.466 μs (0.00% GC)
  mean time:        104.884 μs (15.74% GC)
  maximum time:     44.009 ms (99.52% GC)
  --------------
  samples:          10000
  evals/sample:     1

Porównajmy otrzymany wynik z wynikiem otrzymanym za pomocą biblioteki <tt>GLM</tt>:

In [12]:
using GLM

In [13]:
ols = lm(@formula(MEDV ~ CRIM +ZN + INDUS + CHAS + NOX + RM +
    AGE + DIS + RAD + TAX + PTRATIO + B + LSTAT  + 1), df)

StatsModels.DataFrameRegressionModel{LinearModel{LmResp{Array{Float64,1}},DensePredChol{Float64,LinearAlgebra.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

Formula: MEDV ~ 1 + CRIM + ZN + INDUS + CHAS + NOX + RM + AGE + DIS + RAD + TAX + PTRATIO + B + LSTAT

Coefficients:
               Estimate  Std.Error  t value Pr(>|t|)
(Intercept)     36.6349    5.10204  7.18045   <1e-11
CRIM          -0.107417  0.0328473  -3.2702   0.0012
ZN            0.0461207  0.0137207   3.3614   0.0008
INDUS          0.014269  0.0616534 0.231438   0.8171
CHAS            2.67111   0.861115  3.10192   0.0020
NOX            -17.6336    3.81872 -4.61768    <1e-5
RM              3.79431   0.417835  9.08086   <1e-17
AGE          0.00107616  0.0132048 0.081498   0.9351
DIS            -1.47918   0.199347 -7.42012   <1e-12
RAD            0.301534  0.0663982   4.5413    <1e-5
TAX          -0.0120534 0.00376457 -3.20181   0.0015
PTRATIO       -0.958874   0.130831  -7.3291   <1e-12
B             0.0093054 0.00

Możemy też zdefiniować problem oszacowania parametrów tej regresji  jako zadanie optymalizacyjne:



Zacznijmy od znormalizowania zmiennych:

In [14]:
using StatsBase: mean, std

In [15]:
X = (X .- mean(X, dims = 2)) ./ std(X, dims = 2);

i wyznaczenia początkowych współczynników oraz zdefiniowania funkcji straty $J(\theta)$:

In [16]:
W = rand(1,13);
b = rand();

In [17]:
#predict(W, b, x) = W * x .+ b
Predict(W, b, x) = W * x .+ b
MSE(ŷ, y) = sum((ŷ .- y).^2)/size(y, 2)

MSE (generic function with 1 method)

In [18]:
MSE(Predict(W,b,X), y)

605.0736750758879

Do rozwiązania zadanego problemu wykorzystamy [metodę gradientu prostego](https://pl.wikipedia.org/wiki/Metoda_gradientu_prostego). Zdefiniujmy funkcję, która wyznacza gradient:

In [19]:
function simple_∇(W::Array{Float64}, b::Float64, x::Array{Float64}, 
        y::Array{Float64}; Δ::Float64 = 0.000000001)
    ∇ = Float64[] 
    for i = 1:length(W)
        W′ = W .+ (LinearIndices(W) .== i) * Δ  
        df = (MSE(Predict(W′,b,x),y) - MSE(Predict(W, b ,x),y)) /  Δ
        push!(∇,df)
    end
    ∇b = (MSE(Predict(W,b + Δ,x),y) - MSE(Predict(W,b ,x),y)) /  Δ
    return ∇, ∇b
end

simple_∇ (generic function with 1 method)

In [20]:
simple_∇(W,b,X,y)

([10.0165, -6.87135, 11.2007, -3.40685, 9.55936, -13.8804, 8.35882, -5.47118, 9.88814, 11.5168, 11.2983, -7.36259, 15.7967], -44.65869096748065)

Zdefiniujmy funkcję rozwiązującą zadanie:

In [21]:
function solve(W::Array{Float64}, b::Float64, x::Array{Float64}, y::Array{Float64};
                η::Float64 = 0.001, ϵ::Float64 = 10^-10, maxit::Int64 = 50_000)
    iter = 1
    θ = W
    β = b
    ∇,∇b = simple_∇(W,b,x,y)
    while true
        θ₀, β₀ = θ, β
        θ -= η * ∇'
        β  -= η * ∇b
        ∇,∇b = simple_∇(θ,β,x,y)
        stop = maximum(abs.(hcat(θ, β) .- hcat(θ₀, β₀)))
        stop < ϵ && break
        iter += 1
        iter > maxit && break
    end
    println(iter)
    return θ, β
end

solve (generic function with 1 method)

In [22]:
θ, β = solve(W,b,X,y)

50001


([-0.92448 1.07623 … 0.850192 -3.76692], 22.529901009826464)

In [23]:
MSE(Predict(θ,β,X), y)

21.86558147061406

Problematyczną kwestią jest sposób numerycznego liczenia pochodnych. Klasyczny sposób ich definiowania jest nieefektywny numerycznie. Koniecznie jest wykorzystanie [alternatywnych metod liczenia](http://www.juliadiff.org/). Niestety są one zazwyczaj trudne do samodzielnej implementacji i warto wtedy skorzystać z jednego z zaimplementowanych gdzie indziej rozwiązań.

Rozwiążmy zadanie jeszcze raz, tym razem wykorzystując do tego bibliotekę [<tt>Flux</tt>](https://fluxml.ai/):

In [24]:
using Flux.Tracker, Statistics, DelimitedFiles
using Flux.Tracker: Params, gradient, update!
using Flux: gpu

loaded


In [25]:
W = param(randn(1,13)/10) 
b = param([0.]) 

Tracked 1-element Array{Float64,1}:
 0.0

In [26]:
loss(x, y) = MSE(Predict(W,b,x), y)

loss (generic function with 1 method)

In [27]:
η = 0.001
ϵ =  10^-10
θ = Params([W, b])

Params([[0.0493809 -0.0231421 … -0.0838262 0.0452109] (tracked), [0.0] (tracked)])

In [28]:
while true
    g = gradient(() -> loss(X, y), θ)
    θ₀ = hcat(θ...)
    for x in θ
        update!(x, -g[x]*η)
    end
    @show loss(X,y)
    stop = maximum(abs.(θ₀ .- hcat(θ...)))
    stop < ϵ && break
end

loss(X, y) = 589.5403440857716 (tracked)
loss(X, y) = 586.6398863743763 (tracked)
loss(X, y) = 583.7662007431604 (tracked)
loss(X, y) = 580.9188137041043 (tracked)
loss(X, y) = 578.097262553861 (tracked)
loss(X, y) = 575.3010951146293 (tracked)
loss(X, y) = 572.5298694813056 (tracked)
loss(X, y) = 569.78315377477 (tracked)
loss(X, y) = 567.0605259011577 (tracked)
loss(X, y) = 564.3615733169669 (tracked)
loss(X, y) = 561.6858927998634 (tracked)
loss(X, y) = 559.033090225044 (tracked)
loss(X, y) = 556.4027803470228 (tracked)
loss(X, y) = 553.7945865867076 (tracked)
loss(X, y) = 551.2081408236413 (tracked)
loss(X, y) = 548.6430831932785 (tracked)
loss(X, y) = 546.0990618891773 (tracked)
loss(X, y) = 543.5757329699887 (tracked)
loss(X, y) = 541.0727601711227 (tracked)
loss(X, y) = 538.5898147209807 (tracked)
loss(X, y) = 536.1265751616432 (tracked)
loss(X, y) = 533.682727173901 (tracked)
loss(X, y) = 531.257963406532 (tracked)
loss(X, y) = 528.8519833097101 (tracked)
loss(X, y) = 526.46449

loss(X, y) = 262.9151622411052 (tracked)
loss(X, y) = 261.95015122263885 (tracked)
loss(X, y) = 260.9891452933398 (tracked)
loss(X, y) = 260.0321255270978 (tracked)
loss(X, y) = 259.0790731396972 (tracked)
loss(X, y) = 258.12996948666637 (tracked)
loss(X, y) = 257.1847960611739 (tracked)
loss(X, y) = 256.2435344919708 (tracked)
loss(X, y) = 255.30616654137734 (tracked)
loss(X, y) = 254.37267410331364 (tracked)
loss(X, y) = 253.4430392013728 (tracked)
loss(X, y) = 252.51724398693568 (tracked)
loss(X, y) = 251.59527073732627 (tracked)
loss(X, y) = 250.6771018540066 (tracked)
loss(X, y) = 249.76271986081053 (tracked)
loss(X, y) = 248.85210740221535 (tracked)
loss(X, y) = 247.9452472416501 (tracked)
loss(X, y) = 247.0421222598399 (tracked)
loss(X, y) = 246.1427154531858 (tracked)
loss(X, y) = 245.24700993217817 (tracked)
loss(X, y) = 244.35498891984486 (tracked)
loss(X, y) = 243.46663575023058 (tracked)
loss(X, y) = 242.5819338669095 (tracked)
loss(X, y) = 241.70086682152825 (tracked)
loss

loss(X, y) = 124.11610698744272 (tracked)
loss(X, y) = 123.71459601520138 (tracked)
loss(X, y) = 123.31469752810901 (tracked)
loss(X, y) = 122.9164049963129 (tracked)
loss(X, y) = 122.51971191680465 (tracked)
loss(X, y) = 122.12461181330256 (tracked)
loss(X, y) = 121.73109823613508 (tracked)
loss(X, y) = 121.33916476212453 (tracked)
loss(X, y) = 120.94880499447186 (tracked)
loss(X, y) = 120.56001256264167 (tracked)
loss(X, y) = 120.17278112224828 (tracked)
loss(X, y) = 119.78710435494209 (tracked)
loss(X, y) = 119.40297596829667 (tracked)
loss(X, y) = 119.0203896956966 (tracked)
loss(X, y) = 118.63933929622584 (tracked)
loss(X, y) = 118.2598185545565 (tracked)
loss(X, y) = 117.88182128083878 (tracked)
loss(X, y) = 117.50534131059077 (tracked)
loss(X, y) = 117.1303725045894 (tracked)
loss(X, y) = 116.75690874876183 (tracked)
loss(X, y) = 116.38494395407727 (tracked)
loss(X, y) = 116.01447205643946 (tracked)
loss(X, y) = 115.64548701657999 (tracked)
loss(X, y) = 115.27798281995146 (track

loss(X, y) = 64.3546446941503 (tracked)
loss(X, y) = 64.19241443420421 (tracked)
loss(X, y) = 64.03083094193819 (tracked)
loss(X, y) = 63.86989161507247 (tracked)
loss(X, y) = 63.7095938618479 (tracked)
loss(X, y) = 63.5499351009833 (tracked)
loss(X, y) = 63.39091276163298 (tracked)
loss(X, y) = 63.232524283344056 (tracked)
loss(X, y) = 63.0747671160145 (tracked)
loss(X, y) = 62.91763871985091 (tracked)
loss(X, y) = 62.761136565326694 (tracked)
loss(X, y) = 62.605258133140275 (tracked)
loss(X, y) = 62.45000091417363 (tracked)
loss(X, y) = 62.29536240945084 (tracked)
loss(X, y) = 62.14134013009694 (tracked)
loss(X, y) = 61.98793159729688 (tracked)
loss(X, y) = 61.83513434225454 (tracked)
loss(X, y) = 61.68294590615212 (tracked)
loss(X, y) = 61.53136384010973 (tracked)
loss(X, y) = 61.38038570514472 (tracked)
loss(X, y) = 61.23000907213178 (tracked)
loss(X, y) = 61.08023152176263 (tracked)
loss(X, y) = 60.93105064450635 (tracked)
loss(X, y) = 60.782464040569565 (tracked)
loss(X, y) = 60.

loss(X, y) = 41.65533088443973 (tracked)
loss(X, y) = 41.58326147561527 (tracked)
loss(X, y) = 41.51147668649509 (tracked)
loss(X, y) = 41.43997537647112 (tracked)
loss(X, y) = 41.3687564095297 (tracked)
loss(X, y) = 41.297818654232856 (tracked)
loss(X, y) = 41.2271609836998 (tracked)
loss(X, y) = 41.15678227558842 (tracked)
loss(X, y) = 41.08668141207711 (tracked)
loss(X, y) = 41.01685727984641 (tracked)
loss(X, y) = 40.94730877006068 (tracked)
loss(X, y) = 40.87803477835017 (tracked)
loss(X, y) = 40.80903420479284 (tracked)
loss(X, y) = 40.740305953896346 (tracked)
loss(X, y) = 40.67184893458025 (tracked)
loss(X, y) = 40.60366206015808 (tracked)
loss(X, y) = 40.535744248319624 (tracked)
loss(X, y) = 40.468094421113186 (tracked)
loss(X, y) = 40.400711504927955 (tracked)
loss(X, y) = 40.33359443047659 (tracked)
loss(X, y) = 40.26674213277753 (tracked)
loss(X, y) = 40.20015355113775 (tracked)
loss(X, y) = 40.13382762913534 (tracked)
loss(X, y) = 40.06776331460226 (tracked)
loss(X, y) = 

...

loss(X, y) = 21.901264790643292 (tracked)
loss(X, y) = 21.901255508878105 (tracked)
loss(X, y) = 21.901246229638375 (tracked)
loss(X, y) = 21.90123695292334 (tracked)
loss(X, y) = 21.901227678732248 (tracked)
loss(X, y) = 21.901218407064317 (tracked)
loss(X, y) = 21.901209137918794 (tracked)
loss(X, y) = 21.90119987129493 (tracked)
loss(X, y) = 21.901190607191953 (tracked)
loss(X, y) = 21.9011813456091 (tracked)
loss(X, y) = 21.901172086545625 (tracked)
loss(X, y) = 21.901162830000757 (tracked)
loss(X, y) = 21.901153575973744 (tracked)
loss(X, y) = 21.90114432446381 (tracked)
loss(X, y) = 21.901135075470215 (tracked)
loss(X, y) = 21.901125828992207 (tracked)
loss(X, y) = 21.901116585028998 (tracked)
loss(X, y) = 21.901107343579863 (tracked)
loss(X, y) = 21.90109810464403 (tracked)
loss(X, y) = 21.90108886822073 (tracked)
loss(X, y) = 21.90107963430922 (tracked)
loss(X, y) = 21.901070402908733 (tracked)
loss(X, y) = 21.901061174018523 (tracked)
loss(X, y) = 21.901051947637825 (tracked)


loss(X, y) = 21.89911659966037 (tracked)
loss(X, y) = 21.899107900420358 (tracked)
loss(X, y) = 21.89909920353135 (tracked)
loss(X, y) = 21.899090508992654 (tracked)
loss(X, y) = 21.89908181680357 (tracked)
loss(X, y) = 21.899073126963383 (tracked)
loss(X, y) = 21.89906443947142 (tracked)
loss(X, y) = 21.89905575432697 (tracked)
loss(X, y) = 21.89904707152933 (tracked)
loss(X, y) = 21.899038391077806 (tracked)
loss(X, y) = 21.899029712971714 (tracked)
loss(X, y) = 21.89902103721035 (tracked)
loss(X, y) = 21.899012363793005 (tracked)
loss(X, y) = 21.899003692718995 (tracked)
loss(X, y) = 21.89899502398762 (tracked)
loss(X, y) = 21.898986357598186 (tracked)
loss(X, y) = 21.89897769355 (tracked)
loss(X, y) = 21.89896903184236 (tracked)
loss(X, y) = 21.898960372474573 (tracked)
loss(X, y) = 21.898951715445943 (tracked)
loss(X, y) = 21.898943060755776 (tracked)
loss(X, y) = 21.898934408403388 (tracked)
loss(X, y) = 21.898925758388064 (tracked)
loss(X, y) = 21.89891711070913 (tracked)
loss(X

loss(X, y) = 21.897397913322706 (tracked)
loss(X, y) = 21.897389677341604 (tracked)
loss(X, y) = 21.89738144357458 (tracked)
loss(X, y) = 21.897373212020984 (tracked)
loss(X, y) = 21.897364982680166 (tracked)
loss(X, y) = 21.897356755551485 (tracked)
loss(X, y) = 21.89734853063428 (tracked)
loss(X, y) = 21.8973403079279 (tracked)
loss(X, y) = 21.8973320874317 (tracked)
loss(X, y) = 21.897323869145044 (tracked)
loss(X, y) = 21.897315653067256 (tracked)
loss(X, y) = 21.897307439197714 (tracked)
loss(X, y) = 21.89729922753574 (tracked)
loss(X, y) = 21.897291018080715 (tracked)
loss(X, y) = 21.897282810831978 (tracked)
loss(X, y) = 21.897274605788866 (tracked)
loss(X, y) = 21.897266402950763 (tracked)
loss(X, y) = 21.897258202316998 (tracked)
loss(X, y) = 21.897250003886935 (tracked)
loss(X, y) = 21.897241807659924 (tracked)
loss(X, y) = 21.89723361363531 (tracked)
loss(X, y) = 21.897225421812465 (tracked)
loss(X, y) = 21.897217232190723 (tracked)
loss(X, y) = 21.89720904476945 (tracked)
l

Excessive output truncated after 524328 bytes.

loss(X, y) = 21.896036956809848 (tracked)
loss(X, y) = 21.89602908594867 (tracked)
loss(X, y) = 21.896021217194754 (tracked)
loss(X, y) = 21.89601335054751 (tracked)
loss(X, y) = 21.896005486006313 (tracked)
loss(X, y) = 21.895997623570555 (tracked)
loss(X, y) = 21.895989763239616 (tracked)
loss(X, y) = 21.895981905012892 (tracked)
loss(X, y) = 21.895974048889755 (tracked)
loss(X, y) = 21.895966194869626 (tracked)
loss(X, y) = 21.895958342951843 (tracked)
loss(X, y) = 21.89595049313583 (tracked)
loss(X, y) = 21.89594264542097 (tracked)
loss(X, y) = 21.895934799806646 (tracked)
loss(X, y) = 21.895926956292254 (tracked)


In [29]:
MSE(Predict(θ...,X), y)

21.86557993335148 (tracked)

## Co dalej?

W ramach dzisiejszego prostego przykładu pokazaliśmy sobie to w jaki sposób można budować modele predykcyjne wykorzystując do tego najbardziej podstawowe narzędzia wbudowane w Julię. W celu poszerzenia swojej wiedzy 
Aby być jednak w stanie budować bardziej złożone modele potrzebować będziemy odpowiednich bibliotek ułatwiających nam pracę, między innymi: 

###  1. DataFrames.jl

Biblioteka [<tt>DataFrames</tt>](https://juliadata.github.io/DataFrames.jl/stable/man/getting_started.html) jest narzędziem pozwalającym na efektywną i wygodną pracę ze zbiorami danych. Jest implementacja znanych z <tt>R</tt> ramek danych, oferując wszystkie znane z <tt>R</tt> narzędzia, zaimplementowane w wyraźnie efektywniejszy obliczeniowo sposób. Warto zapoznać się ze szczegółowym [wprowadzeniem do <tt>DataFrames</tt> ](https://github.com/bkamins/Julia-DataFrames-Tutorial).

### 2. Plots.jl

[Plots]() to podstawowa bibliotka do tworzenia wykresów w Julii. Jedną z jej głównych zalet jest to, że pozwala na wykorzystanie wielu [backendów](http://docs.juliaplots.org/latest/backends/). Warto zapoznać się z [dokumenacją](http://docs.juliaplots.org/latest/) tej biblioteki.

### 3. Flux.jl

W celu zapoznania się z działaniem biblioteki <tt>Flux</tt> warto zajrzeć do jej [dokumentacji](https://fluxml.ai/Flux.jl/stable/) i zapoznać się z załączonym do dzisiejszych notatek skryptem.

## Literatura 

Poza wymienioną powyżej literaturą wprowadzającą do specyficznych zagadnień potrzebnych podczas tego kursu istenieje wiele dodatkowych przydatnych źródeł pozwalających rozwinąć swoją wiedzę na temat programowania w Julii:
- [Julia Express](https://github.com/bkamins/The-Julia-Express)
- [wykłady Quantitative Economics Sargenta i Stachurskiego](https://lectures.quantecon.org/jl/)
- [Julia dla Data Science](http://ucidatascienceinitiative.github.io/IntroToJulia/)
- [Think Julia](https://benlauwens.github.io/ThinkJulia.jl/latest/book.html)
- [materiały dostępne na stronie języka](https://julialang.org/learning/)

## Dodatkowe prace domowe

1. Przepisz zaprezentowaną powyżej implementacje klasycznej metody najmniejszych kwadratów na języki <tt>Python</tt> i <tt>R</tt>. Porównaj czasy rozwiązania problemu.

2. Zaprezentowana na zajęciach implementacja estymacji parametrów modelu za pomocą klasycznej metody najmniejszych kwadratów nie jest efektywnym numerycznie rozwiązaniem problemu. Postaraj się zmodyfikować je w taki sposób aby było szybsze niż rozwiązanie prezentowane na zajęciach. Porównaj oba sposoby zaimplementowania algorytmu.